# Results of complementation of Büchi automata
This notebook presents the data used for Section 4.2 of the CAV paper. The cells `[4]` and `[5]` contain data for **Table 3** from the paper; the table is shown in cell `[6]`. 

In [1]:
from ltlcross_wrapper import ResAnalyzer, gather_cumulative, gather_mins
import pandas as pd
pd.set_option("precision",0)

import spot
spot.setup()
from spot.jupyter import display_inline

from tools import benchmark_names as names

### Visualization of the cumulative data over all benchmarks
For each benchmark, we list the cumulative numbers of states for each tool. The best value for each benchmark is highlighted by green background. The benchmarks start with translation of either `random` formulas or formulas from `literature` by `ltl2tgba`. The suffix `_det` indicates that `ltl2tgba` created automata, that are already deterministic, `_sd` stands for semi-deterministic (but not deterministic), and `_nd` represent automata that are not even semi-deterministic.

The tools are:

* `spot` : `autfilt` from [Spot](https://spot.lrde.epita.fr/) library performs determinization-based complementation; it is configured to produce TGBA as an output.
* `roll` : `Buechic` from [ROLL](https://iscasmc.ios.ac.cn/roll/doku.php) library which is based on automata learning techniques.
* `goal#fri` : The Fribourg complementation plugin for [GOAL](http://goal.im.ntu.edu.tw) which is based on [this paper](https://dl.acm.org/doi/10.1145/3209108.3209138).
* `goal#pit` : The Piterman complementation of GOAL which is based on Piterman's determinization (variant of Safra's construction) and conversion to NBA.
* `seminator#best` : The default complementation in [Seminator 2](https://github.com/mklokocka/seminator). The workflow here is that Seminator first performs semi-determinization and then applies two transition-based variants of the [NCSB algorithm](https://www.fi.muni.cz/~xstrejc/publications/tacas2016coSDBA_preprint.pdf) for complementation of semi-deterministic automata. The smaller of the results is then returned. The two variants are:
  - `spot` the algorithm as was already implemented in Spot
  - `pldi` the new version of the algorithm implemented in Seminator, it is based on this [PLDI'18 paper](https://dl.acm.org/doi/10.1145/3192366.3192405).

The `yes` prefix in the tool names means that the Spot simplifications were applied on the results of the tools (were not disabled for seminator).

The precomputed results contain also data without the simplifications of Spot and result of some other tool configurations that were not presented in the paper. You can control what is shown in cell `[3]` by changing the variable `tools`. The data without Spot's simplifications can be displayed by changing the `yes` prefix in the tool names to `no` (as performed down in this notebook). 

By setting the cell `[2]` to
```python
tools = None
```
you can display results for all benchmarked configurations. On top of (both `yes` and `no` versions of the tools described above, you will find resutls for the following tools.

* `spot_DPA` : `autfilt` configured to return parity automata without the conversion to TGBA
* `seminator#spot` and `seminator#pldi` are the two variants of the complementation in Seminator. Here we call the selected variant exclusively.

In [2]:
tool_set = ["yes.roll","yes.goal#fri","yes.goal#pit","yes.spot","yes.seminator#best"]

In [3]:
benchmarks = {}
for name in names:
    b = ResAnalyzer(f"data/{name}.csv", tool_set=tool_set, cols=["states","time","acc","transitions","edges"])
    b.name = name
    b.orig_count = len(b.values)
    b.clean_count = len(b.values.dropna())
    benchmarks[name] = b

## Cumulative number of states
Cumulative number of automata sizes for each benchmark. The main part of **Table 3** from the paper.

In [4]:
gather_cumulative(benchmarks)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.goal#fri,627,290,142,2530,3294,5278
yes.goal#pit,617,277,206,2490,3676,7713
yes.roll,1388,833,272,3687,5681,6225
yes.seminator#best,622,210,169,2511,2781,4919
yes.spot,611,190,181,2477,2829,5310


### Minimal automata

The following table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools selected by `tool_set` from cell `[2]`. The number in the column **min hits** shows how many times the same size as the smallest automaton was achieved. The number in **unique min hits** counts only cases where the given tool is the only tool with such a small automaton.

In [5]:
gather_mins(benchmarks)

## Table 3

In [6]:
cum = gather_cumulative(benchmarks, highlight=False).loc[tool_set]
mins = gather_mins(benchmarks, highlight=False, unique_only=False).loc[tool_set]
mins.columns=mins.columns.droplevel(1)
df = pd.DataFrame()

for c in cum.columns:
    df[c] = cum[c].astype(int).astype(str)
    df[c] = df[c].str.cat(mins[c].astype(int).astype(str), sep=" (", join="left") + ")"
    
# Add formula counts
counts = {name: f"{b.clean_count}+{b.orig_count - b.clean_count}" for name, b in benchmarks.items()}
counts = pd.Series(counts, name="# of formulas")
df = pd.DataFrame(counts).transpose().append(df.loc[tool_set])

df.columns = pd.MultiIndex.from_tuples([c.split("_") for c in df.columns]).swaplevel()
df = df.sort_index(axis=1)
df = df[["det","sd","nd"]]

tool_names={
    "yes.spot" : "Spot",
    "yes.roll" : "ROLL+Spot",
    "yes.goal#fri": "Fribourg+Spot",
    "yes.goal#pit": "GOAL+Spot",
    "yes.seminator#best": "Seminator 2"
}
columns = {
    "literature" : "literature",
    "random" : "random",
    "det" : "deterministic",
    "sd" :  "semi-detereministic",
    "nd" : "non-semi-deterministic",
}
df.rename(index=tool_names, columns=columns)

deterministic             semi-detereministic              \
                 literature      random          literature      random   
# of formulas         147+5       500+0                47+2       499+1   
ROLL+Spot          1388 (0)    3687 (0)             833 (0)    5681 (4)   
Fribourg+Spot     627 (137)  2530 (464)            290 (26)  3294 (258)   
GOAL+Spot         617 (143)  2490 (477)            277 (28)  3676 (125)   
Spot              611 (150)  2477 (489)            190 (40)  2829 (354)   
Seminator 2       622 (142)  2511 (465)            210 (37)  2781 (420)   

              non-semi-deterministic              
                          literature      random  
# of formulas                   15+5      486+14  
ROLL+Spot                    272 (0)   6225 (58)  
Fribourg+Spot               142 (14)  5278 (238)  
GOAL+Spot                    206 (5)   7713 (96)  
Spot                         181 (9)  5310 (202)  
Seminator 2                  169 (8)  4919 (277)

### Cross-comparison by automata type
The cross-comparison for a benchmark shows, in a cell (`row`,`column`) in how many cases the tool in `row` produces automaton that is better thatn the one produced by `column`. The last columns (`V`) summs the numbers across rows, while the green highlighting fill a space that is proportional to how well the tool in `row` competed agains `column` (proportional across columns).

In [7]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(gather_cumulative(bench, tool_set))
    for b in bench.values():
        display(b.name, b.cross_compare(tool_set, include_fails=True))
    print("\n\n")

,literature_nd,random_nd
tool,,
yes.goal#fri,142,5278
yes.goal#pit,206,7713
yes.roll,272,6225
yes.seminator#best,169,4919
yes.spot,181,5310


'literature_nd'

,yes.roll,yes.goal#fri,yes.goal#pit,yes.spot,yes.seminator#best,V
yes.roll,nan,1,4,2,2,9
yes.goal#fri,19,nan,15,12,9,55
yes.goal#pit,16,2,nan,9,3,30
yes.spot,18,5,10,nan,7,40
yes.seminator#best,18,6,12,12,nan,48


'random_nd'

,yes.roll,yes.goal#fri,yes.goal#pit,yes.spot,yes.seminator#best,V
yes.roll,nan,113,231,124,96,564
yes.goal#fri,387,nan,379,233,228,1227
yes.goal#pit,266,58,nan,111,102,537
yes.spot,375,221,368,nan,213,1177
yes.seminator#best,402,242,381,243,nan,1268


...

,literature_det,random_det
tool,,
yes.goal#fri,627,2530
yes.goal#pit,617,2490
yes.roll,1388,3687
yes.seminator#best,622,2511
yes.spot,611,2477


'literature_det'

,yes.roll,yes.goal#fri,yes.goal#pit,yes.spot,yes.seminator#best,V
yes.roll,nan,2,2,0,4,8
yes.goal#fri,150,nan,5,15,13,183
yes.goal#pit,150,6,nan,15,11,182
yes.spot,152,15,12,nan,14,193
yes.seminator#best,148,8,4,14,nan,174


'random_det'

,yes.roll,yes.goal#fri,yes.goal#pit,yes.spot,yes.seminator#best,V
yes.roll,nan,1,1,0,5,7
yes.goal#fri,499,nan,36,21,59,615
yes.goal#pit,499,20,nan,13,50,582
yes.spot,500,43,44,nan,64,651
yes.seminator#best,495,27,13,9,nan,544


### Running times (in seconds) and timeouts
The complementation implemented in Seminator 2 is also very competitive in terms of running times.

In [8]:
gather_cumulative(benchmarks, col="time")

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.goal#fri,820,393,276,1791,2097,3102
yes.goal#pit,807,536,382,1821,2425,4302
yes.roll,1036,213,594,860,1034,2882
yes.seminator#best,7,2,2,19,23,357
yes.spot,7,2,12,19,22,147


In [9]:
for name, b in benchmarks.items():
    print(name)
    display(b.get_error_counts())
    print("\n")

literature_det


,timeout,parse error,incorrect,crash,no output
no.roll,4,0,0,1,0
yes.roll,4,0,0,0,0




literature_sd


,timeout,parse error,incorrect,crash,no output
no.goal#fri,2,0,0,0,0
no.goal#pit,2,0,0,0,0
no.roll,1,0,0,0,0
yes.goal#fri,1,0,0,0,0
yes.goal#pit,2,0,0,0,0
yes.roll,1,0,0,0,0




literature_nd


...

,timeout,parse error,incorrect,crash,no output
tool,,,,,




random_sd


,timeout,parse error,incorrect,crash,no output
no.goal#pit,1,0,0,0,0
yes.goal#pit,1,0,0,0,0




random_nd


,timeout,parse error,incorrect,crash,no output
no.goal#fri,3,0,0,0,0
no.goal#pit,6,0,0,0,0
no.roll,5,0,0,0,0
yes.goal#fri,3,0,0,0,0
yes.goal#pit,8,0,0,0,0
yes.roll,5,0,0,0,0
yes.seminator#best,1,0,0,0,0
yes.seminator#pldi,1,0,0,0,0
yes.seminator#spot,1,0,0,0,0
yes.spot,1,0,0,0,0


### Scatter plots
We offer interactive scatter plots that show how Seminator 2 compares to Spot (`In[10]`) and to the Fribourg complementation with Spot's simplifications (`In[13]`) on the _not semi-deterministic random_ benchmark. In the paper, we present them as **Figure 5**. We also show comparison to the determinization-based complementation from GOAL (with Spot's simplifications), `yes.goal#pit` (`In[14]`) that is not presented in the paper.

When you click on some of the dots, you will get instructions on how to dispaly the formulas represented by the dot (some variant of cell `[11]`. Using the `formula_id` you can further display the two automata (we advise to do this only for automata of reasonable size for obvious reasons) as demonstrated in cell `[12]`. We cannot display the automata for `owl#best` as this is just a virtual tool for which we compute values (like number of states, time, etc.). The automaton is produced always by one of `yes.owl#a` or `yes.owl#s`.

In [10]:
b = benchmarks["random_nd"]

In [11]:
b.bokeh_scatter_plot("yes.spot","yes.seminator#best")

Loading BokehJS ...

Figure(id='1002', ...)

In [12]:
data = b.get_plot_data('yes.spot','yes.seminator#best',add_count=False)
data[(data['yes.spot'] == 2) & (data['yes.seminator#best'] == 4)]

,tool,yes.spot,yes.seminator#best
form_id,formula,,
189,F(FGa R b),2,4


In [13]:
tool1 = 'yes.spot'
tool2 = 'yes.seminator#best'
form_id = 189
display_inline(b.aut_for_id(form_id, tool1), b.aut_for_id(form_id, tool2))

In [14]:
b.bokeh_scatter_plot("yes.goal#fri","yes.seminator#best")

Loading BokehJS ...

Figure(id='1124', ...)

In [15]:
p = b.bokeh_scatter_plot("yes.goal#pit","yes.seminator#best")

Loading BokehJS ...

## Without simplifications of Spot
GOAL#pit runs removing dead and unreachable states, roll probably does not create such states. The rest of the tools does not remove them.

We can observe that Fribourg generates large ammount of unnecessary states before simplifications.

In [16]:
no_tools = [t.replace("yes","no") for t in tool_set]

In [17]:
gather_cumulative(benchmarks, tool_set=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.goal#fri,1251,2518,642,5135,15437,48796
no.goal#pit,767,1108,369,2992,8954,22941
no.roll,1628,915,271,4775,6799,7486
no.seminator#best,798,439,470,3100,5775,18341
no.spot,608,257,205,2480,3497,6974


## Minimal automata
The Fribourg complementation never returns automaton with the minimal number of states and thus is removed from the next table.

In [18]:
gather_mins(benchmarks, tool_set=no_tools)

### Time in seconds

In [19]:
gather_cumulative(benchmarks, col="time", tool_set=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.goal#fri,829,444,271,1795,2089,3056
no.goal#pit,783,549,395,1784,2418,4142
no.roll,997,214,664,858,1026,2842
no.seminator#best,6,2,1,19,22,31
no.spot,6,1,1,18,21,24


In [20]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(t, gather_cumulative(bench, no_tools))
    for b in bench.values():
        display(b.cross_compare(no_tools))
    print("\n\n\n")

'nd'

,literature_nd,random_nd
tool,,
yes.goal#fri,142,5278
yes.goal#pit,206,7713
yes.roll,272,6225
yes.seminator#best,169,4919
yes.spot,181,5310


,no.roll,no.goal#fri,no.goal#pit,no.spot,no.seminator#best,V
no.roll,nan,16,14,3,15,48
no.goal#fri,4,nan,1,0,3,8
no.goal#pit,6,18,nan,1,9,34
no.spot,17,20,19,nan,19,75
no.seminator#best,5,17,10,1,nan,33


,no.roll,no.goal#fri,no.goal#pit,no.spot,no.seminator#best,V
no.roll,nan,473,412,134,287,1306
no.goal#fri,27,nan,17,0,25,69
no.goal#pit,88,480,nan,15,101,684
no.spot,365,500,484,nan,442,1791
no.seminator#best,213,475,397,57,nan,1142


'sd'

...

,no.roll,no.goal#fri,no.goal#pit,no.spot,no.seminator#best,V
no.roll,nan,391,255,28,131,805
no.goal#fri,107,nan,8,0,0,115
no.goal#pit,237,490,nan,1,10,738
no.spot,471,500,496,nan,372,1839
no.seminator#best,369,500,480,50,nan,1399


'det'

,literature_det,random_det
tool,,
yes.goal#fri,627,2530
yes.goal#pit,617,2490
yes.roll,1388,3687
yes.seminator#best,622,2511
yes.spot,611,2477


,no.roll,no.goal#fri,no.goal#pit,no.spot,no.seminator#best,V
no.roll,nan,60,2,0,7,69
no.goal#fri,91,nan,0,0,0,91
no.goal#pit,150,152,nan,0,76,378
no.spot,152,152,113,nan,141,558
no.seminator#best,144,150,56,0,nan,350


,no.roll,no.goal#fri,no.goal#pit,no.spot,no.seminator#best,V
no.roll,nan,292,7,0,16,315
no.goal#fri,203,nan,0,0,2,205
no.goal#pit,493,500,nan,0,135,1128
no.spot,500,500,431,nan,448,1879
no.seminator#best,483,496,75,0,nan,1054


In [21]:
for name, b in benchmarks.items():
    display(name, b.get_error_counts())

'literature_det'

,timeout,parse error,incorrect,crash,no output
no.roll,4,0,0,1,0
yes.roll,4,0,0,0,0


'literature_sd'

,timeout,parse error,incorrect,crash,no output
no.goal#fri,2,0,0,0,0
no.goal#pit,2,0,0,0,0
no.roll,1,0,0,0,0
yes.goal#fri,1,0,0,0,0
yes.goal#pit,2,0,0,0,0
yes.roll,1,0,0,0,0


'literature_nd'

...

,timeout,parse error,incorrect,crash,no output
tool,,,,,


'random_sd'

,timeout,parse error,incorrect,crash,no output
no.goal#pit,1,0,0,0,0
yes.goal#pit,1,0,0,0,0


'random_nd'

,timeout,parse error,incorrect,crash,no output
no.goal#fri,3,0,0,0,0
no.goal#pit,6,0,0,0,0
no.roll,5,0,0,0,0
yes.goal#fri,3,0,0,0,0
yes.goal#pit,8,0,0,0,0
yes.roll,5,0,0,0,0
yes.seminator#best,1,0,0,0,0
yes.seminator#pldi,1,0,0,0,0
yes.seminator#spot,1,0,0,0,0
yes.spot,1,0,0,0,0
